In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np
import json
import random
from time import time

In [ ]:
time1 = time()
'''
parser converts string to list of dicts
ast -> literal_eval can be used too but json.loads is 10 times faster
[:] is not the right way to copy a df. it might create a view etc since applying a func affects both but adding a column doesn't
'''
def parser2(data):
    if data != data:
        return np.nan
    else:
        return json.loads(data)
dft=pd.read_csv("JCO5003.tsv", dtype = {'srv2' : object}, sep ='\t')
dft.columns = dft.columns.str.replace('d\.', '', regex=True)
dft.dt= pd.to_datetime(dft.dt, format = '%d-%m-%y %H:%M:%S:%f')
#dft.jbdl_ts= pd.to_datetime(dft.jbdl_ts, infer_datetime_format=True)
#dft.wi17 = dft.wi17.apply(parser2)
#dft.wi18 = dft.wi18.apply(parser2)
#df = df[df['di8'].isin(random.sample(set(df[df['partition_date'] == df.partition_date.unique()[-1]].di8.unique()), 50))]
dft= dft.sort_values(by = [ 'dt']).reset_index(drop=True)

In [ ]:
df=dft.copy()
df['di4'] = 'JCO500'
df['di5'] = 'R1.192'
df['partition_date'] = '2021-04-29'

In [ ]:
# bdf1 = df.loc[:df.shape[0]//5][:]
# bdf2 = df.loc[df.shape[0]//5 : 2*df.shape[0]//5][:]
# bdf3 = df.loc[2*df.shape[0]//5 : 3*df.shape[0]//5][:]
# bdf4 = df.loc[3*df.shape[0]//5 : 4*df.shape[0]//5][:]
# bdf5 = df.loc[4*df.shape[0]//5 :][:]

In [ ]:
# %store -r bdf1
# %store -r bdf2
# %store -r bdf3
# %store -r bdf4
# %store -r bdf5

In [ ]:
# df = pd.concat([bdf1, bdf2, bdf3, bdf4, bdf5])
#df=df.loc[df.di5 == 'R1.130'].reset_index(drop=True)

In [ ]:
pd.set_option('display.max_columns', None, 'display.max_rows', 500)
pd.set_option('display.width', 140)
pd.set_option('display.memory_usage', True, 'compute.use_numba', False)
plt.rcParams.update({'font.size': 12})
n=1
print(f'\t\t\t\t\t{df.di4.unique()[0]}_{df.di5.unique()} Analysis for {df.partition_date.unique()[0]} to {df.partition_date.unique()[-1]}')

In [ ]:
datewisedi8 = df.groupby(['partition_date'])['di8'].apply(lambda x: set(x.values)).to_dict()

In [ ]:
fig, ax = plt.subplots(2,figsize=(18,12))
ax[0].bar(df.groupby('si2')['di8'].count().reset_index()['si2'], df.groupby('si2')['di8'].count().reset_index()['di8'], color = 'c')
ax[0].set_xticks(range(0,102,2))
ax[0].set_ylabel("Count of an instance of CPU utilisation")
ax[0].set_xlabel("CPU Utilisation value")
ax[0].set_title(f'[1] : {df.di4.unique()[0]}_{df.di5.unique()[0][1:]} CPU Utilisation stats')
ax[1].bar(df.groupby('si3')['di8'].count().reset_index()['si3'], df.groupby('si3')['di8'].count().reset_index()['di8'], color = 'g')
ax[1].set_xticks(range(0,102,2))
ax[1].set_ylabel("Count of an instance of Memory utilisation")
ax[1].set_xlabel("Memory Utilisation value")
ax[1].set_title(f'[2] : {df.di4.unique()[0]}_{df.di5.unique()[0][1:]} Memory Utilisation stats')
plt.locator_params(axis='y', nbins=10)
fig.tight_layout(pad=3.0)

In [ ]:
def cpuflag(d):
    return sum(d>70) >= 0.2*len(d)
l=df.groupby(['partition_date', 'di8' ])['si2'].apply(cpuflag).reset_index()
print('[3] : Date-wise High CPU utilisation Flags:\n')
l.groupby('partition_date').agg({'di8' : 'count', 'si2' : 'sum'}).rename(columns = {'di8' : 'Total No. of devices', 'si2' : 'Number of devices with high CPU utilisation'})
# for i in df.partition_date.unique():
#     tv = len(l[l['si2'] & (l['partition_date'] == i)]['di8'])
#     print(f"\t● {None if not tv else tv } of the {len(l[l['partition_date'] == i]['di8'])} devices reported HIGH CPU Usage on {i} :\n")
#     if len(l[l['si2'] & (l['partition_date'] == i)]['di8']):
#         print(f"\t\t{l[l['si2'] & (l['partition_date'] == i)]['di8'].tolist()}\n")

In [ ]:
df.loc[df.di8 == 'RUFOTCC00081776']['si2'].plot(figsize = (18,5))

In [ ]:
l2=df.groupby(['partition_date', 'di8' ])['si3'].apply(cpuflag).reset_index()
print('[4] : Date-wise High Memory utilisation Flags:\n')
l2.groupby('partition_date').agg({'di8' : 'count', 'si3' : 'sum'}).rename(columns = {'di8' : 'Total No. of devices', 'si3' : 'Number of devices with high Memory utilisation'})
# for i in df.partition_date.unique():
#     tv = len(l2[l2['si3'] & (l2['partition_date'] == i)]['di8'])
#     print(f"\t● {None if not tv else tv } of the {len(l2[l2['partition_date'] == i]['di8'])} devices reported HIGH Memory Usage on {i} : \n")
#     if len(l2[l2['si3'] & (l2['partition_date'] == i)]['di8']):
#         print(f"\t\t{l2[l2['si3'] & (l['partition_date'] == i)]['di8'].tolist()}\n")

In [ ]:
def rebootcounter(d):
    return sum(d.diff()<0)
r2 = df.groupby(['partition_date', 'di8' ])['pi1'].apply(rebootcounter).reset_index().rename(columns = {'partition_date' : 'Date', 
                                                                                                        'pi1' : 'NoR',
                                                                                                       'di8' : 'RSN'})
try:
    his = np.histogram(r2['NoR'],bins=range( max(r2[r2['NoR'] > 2][ 'NoR']) + 2))
    fig, ax = plt.subplots(figsize=(12,5))
    ax.set_xlabel("Number of Reboots in a day")
    ax.set_ylabel("Count of devices")
    #ax.set_yticks(range(0,max(his[0][3:])))
    ax.set_xticks(range(3,max(his[1])))
    ax.set_title("[5] : Count of devices vs Number of reboots in a day")
    rects = ax.bar(his[1][3:-1], his[0][3:], color = 'crimson', align='center')
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
except:
    print('[5]: No multiple reboots found.')

In [ ]:
print("\n[6] : Top 5 corner cases for number of reboots in a day :")
r2.sort_values('NoR', ascending=False).reset_index(drop=True)[:6]

In [ ]:
print("\n[6] : Top 5 corner cases for number of reboots in a day :")
r2.sort_values('NoR', ascending=False).reset_index(drop=True)[:]

In [ ]:
def timediff(d):
    return(max(d['jbdl_ts']-d['dt']))
df2=df.groupby(['partition_date', 'di8' ])[['dt', 'jbdl_ts']].apply(timediff).reset_index().rename(columns = {0:'Max diff'})
df2['plot'] = df2['Max diff'].dt.ceil('20min')
df2 = df2[df2['plot'] < '23:00:00.000000'].reset_index(drop=True)
df3 = df2.groupby('plot')['di8'].count().reset_index()
df3 = df3[df3['plot'] > '0 days 00:20:00']
fig, ax = plt.subplots(figsize=(25,8))
ax.set_xlabel("Upto (hrs:mins:sec) late on a given day")
ax.set_ylabel("Count of devices")
ax.set_title("[7] : Data publish delay vs Count of devices")
rects = ax.bar(df3['plot'].astype(str).replace('0 days ', '', regex=True), df3['di8'], color= 'coral')
for rect in rects:
    height = rect.get_height()
    ax.annotate(f'{height}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [ ]:
print("\n[8] : Top 5 corner cases from the above plot :")
df2.sort_values('Max diff', ascending = False).reset_index(drop=True)[:6]

In [ ]:
def channel(d):
    return((d['wi20'].diff().dropna() !=0 ) & (~(d['pi1'].diff().dropna() < 0)))
df2 = df.groupby(['partition_date', 'di8' ]).apply(channel).reset_index().drop('level_2', axis =1).rename(columns = {0:'change'})
print('[9] : Devices that switched channels in 2.4GHz band without a reboot')
df2[df2['change']].groupby(['partition_date', 'di8'])['change'].sum().to_frame().rename(columns = {'change' : 'Number of times the ONT switched channels'})

In [ ]:
print('[10] : Devices that reported 2.4 GHz channel as 0')
df[df['wi20'] == 0][['di8', 'dt']]

In [ ]:
#if 'JCO110' != df.di4.unique()[0]: 
def channel(d):
    return((d['wi28'].diff().dropna() !=0 ) & (~(d['pi1'].diff().dropna() < 0)))
df2 = df.groupby(['partition_date', 'di8' ]).apply(channel).reset_index().drop('level_2', axis =1).rename(columns = {0:'change'})
print('[11] : Devices that switched channels in 5GHz band without a reboot')
df2[df2['change']].groupby(['partition_date', 'di8'])['change'].sum().to_frame().rename(columns = {'change' : 'Number of times the ONT switched channels'})

In [ ]:
#if 'JCO110' != df.di4.unique()[0]: 
print('[12] : Devices that reported 5 GHz channel as 0')
df[df['wi28'] == 0][['di8', 'dt']]

In [ ]:
def router(d):
    d['boolval'] = (d.li1.sum() != 0) & ((d['wi27'].sum() + d.wi19.sum()) == 0)
    d['Average Ethernet clients'] = round(d['li1'].mean())
    return d[['di8', 'boolval', 'Average Ethernet clients']]
df2 = df.groupby('di8').apply(router).reset_index()
df2.drop(columns = 'index', inplace =True)
df3 =  df2[df2['boolval']].drop_duplicates()
#datewisedi8 = {k: v-set(df3.di8) for k,v in datewisedi8.items()}
print(f'[13] : A total of [{len(df3)}] devices from the sample are being used as a Ethernet only devices. Possibly inidicating pairing with another router/AP. \n')
print('[14] : Following are a few devices with max number of ethernet clients:')
df3[['di8', 'Average Ethernet clients']].sort_values('Average Ethernet clients', ascending = False).reset_index(drop=True)[:6]

In [ ]:
def emptyband2(d):
    return((d['wi19'].sum() != 0) & (d['wi27'].sum() == 0))
df2 =df.groupby('di8').apply(emptyband2).reset_index()
#datewisedi8_2_4 = {k: len(v-set(df2[df2[0]].di8)) for k,v in datewisedi8.items()}
if 'JCO110' != df.di4.unique()[0]:
    print(f'[15] : A total of {len(df2[df2[0]])} devices from the sample are only using the 2.4GHz band. \n')
    #print(tabulate(df2[df2[0]].reset_index(drop= True), headers='keys', tablefmt='psql'), '\n')

In [ ]:
if 'JCO110' != df.di4.unique()[0]: 
    def emptyband5(d):
        return((d['wi19'].sum() == 0) & (d['wi27'].sum() != 0))
    df2 =df.groupby('di8').apply(emptyband5).reset_index()
    #datewisedi8_5 = {k: len(v-set(df2[df2[0]].di8)) for k,v in datewisedi8.items()}
    print(f'[16] : A total of {len(df2[df2[0]])} devices from the sample are only using the 5GHz band. \n')
    #print(tabulate(df2[df2[0]].reset_index(drop= True), headers='keys', tablefmt='psql'), '\n')

In [ ]:
#df2 = df.groupby(['partition_date', 'di8']).apply(lambda x : sum(x.li1) ==0 ).reset_index()
#datewisedi8_e = {k: len(v-set(df2[df2[0]].di8)) for k,v in datewisedi8.items()}
dates = len(df.partition_date.unique())

In [ ]:
def ethererror(d):
    return((sum(d['li6']/d['li4'] > 0.1)/(len(d.li4.replace(0,np.nan).dropna()) or np.nan)) > 0.2 ,
           (sum(d.li7/d.li5 > 0.1)/(len(d.li5.replace(0,np.nan).dropna()) or np.nan)) > 0.2 ,
           (sum(d['li8']/d['li4'] > 0.1)/(len(d.li4.replace(0,np.nan).dropna()) or np.nan)) > 0.2 ,
           (sum(d['li9']/d['li5'] > 0.1)/(len(d.li4.replace(0,np.nan).dropna()) or np.nan)) > 0.2)

df8=df.groupby(['partition_date', 'di8']).apply(ethererror).reset_index()
#df8=df8.drop(columns = [0])
df8[['Tx', 'Rx', 'DTx', 'DRx']] = pd.DataFrame(df8[0].tolist(), index= df8.index)
l=['High Tx\n error pkts', 'High Rx\n error pkts', 'High Tx pkt\n drop rates', 'High Rx pkt\n drop rates']
def plotfunc(d):
    return(len(d),len(d[d['Tx']]),len(d[d['Rx']]),len(d[d['DTx']]),len(d[d['DRx']]))

t = df8.groupby(['partition_date']).apply(plotfunc)
#r=[[0 if ~datewisedi8_e[i] else round(j/datewisedi8_e[i]*100, 2) for j in t[i]] for i in datewisedi8_e.keys()]
fig, ax = plt.subplots((dates+1)//2,2, figsize=(18,(dates+1)//2 *7))
# ax[0,0].bar(l,r[0])
# ax[0,1].bar(l,r[1])
# ax[1,0].bar(l,r[2])
# ax[1,1].bar(l,r[3])
j=0
for x in ax:
    for s in x:
        if j < dates:
            rects = s.bar(l,[t[j][1]*100//t[j][0], t[j][2]*100//t[j][0], t[j][3]*100//t[j][0], t[j][4]*100//t[j][0]])
            s.set_title(f'[17.{j}]: Ethernet stats on {sorted([*datewisedi8])[j]}')
            s.set_ylabel('Percentage of total valid devices(%)')
            s.set_yticks(range(0,101,10))  
            for rect in rects:
                height = rect.get_height()
                s.annotate(f'{height}%',
                                xy=(rect.get_x() + rect.get_width() / 2, height),
                                xytext=(0, 3),  # 3 points vertical offset
                                textcoords="offset points",
                                ha='center', va='bottom')
            j+=1

In [ ]:
def wifierror(d):
    return (sum(d.wi25/d.wi23 > 0.35)/(len(d.wi23.replace(0,np.nan).dropna()) or np.nan)) > 0.2 , (sum(d.wi26/d.wi24 > 0.2)/(len(d.wi24.replace(0,np.nan).dropna()) or np.nan)) > 0.2 
df8=df.groupby(['partition_date', 'di8']).apply(wifierror).reset_index()
df8[['Tx', 'Rx']] = pd.DataFrame(df8[0].tolist(), index= df8.index)
l=['High Tx\n Error rates', 'High Rx\n Error rates']
def plotfunc(d):
    return(len(d), len(d[d['Tx']]),len(d[d['Rx']]))
t = df8.groupby(['partition_date']).apply(plotfunc)
#r=[[0 if ~datewisedi8_2_4[i] else round(j/datewisedi8_2_4[i]*100, 2) for j in t[i]] for i in datewisedi8_2_4.keys()]
fig, ax = plt.subplots((dates+1)//2,2, figsize=(18,(dates+1)//2 *6))
j=0
for x in ax:
    for s in x:
        if j < dates:
            rects = s.bar(l,[t[j][1]*100//t[j][0], t[j][2]*100//t[j][0]])
            s.set_title(f'[18.{j}]: 2.4 GHz band stats on {sorted([*datewisedi8])[j]}')
            s.set_ylabel('Percentage of total valid devices(%)')
            s.set_yticks(range(0,101,10))  
            for rect in rects:
                height = rect.get_height()
                s.annotate(f'{height}%',
                                xy=(rect.get_x() + rect.get_width() / 2, height),
                                xytext=(0, 3),  # 3 points vertical offset
                                textcoords="offset points",
                                ha='center', va='bottom')
            j+=1

In [ ]:
'''
short circuiting used to deal with divide by zero errors
'''
if 'JCO110' != df.di4.unique()[0]:    
    def wifierror(d):
        return (sum(d.wi15/d.wi13 > 0.35)/(len(d.wi13.replace(0,np.nan).dropna()) or np.nan)) > 0.2 , (sum((d.wi16/d.wi14) > 0.35)/(len(d.wi14.replace(0,np.nan).dropna()) or np.nan)) > 0.2
    df8=df.groupby(['partition_date', 'di8']).apply(wifierror).reset_index()
    df8[['Tx', 'Rx']] = pd.DataFrame(df8[0].tolist(), index= df8.index)
    l=['High Tx\n Error rates', 'High Rx\n Error rates']
    t = df8.groupby(['partition_date']).apply(plotfunc)
    #r=[[0 if ~datewisedi8_5[i] else round(j/datewisedi8_5[i]*100, 2) for j in t[i]] for i in datewisedi8_5.keys()]
    fig, ax = plt.subplots((dates+1)//2,2, figsize=(18,(dates+1)//2 *6))
    j=0
    for x in ax:
        for s in x:
            if j < dates:
                rects = s.bar(l,[t[j][1]*100//t[j][0], t[j][2]*100//t[j][0]])
                s.set_title(f'[19.{j}]: 5 GHz band stats on {sorted([*datewisedi8])[j]}')
                s.set_ylabel('Percentage of total valid devices(%)')
                s.set_yticks(range(0,101,10))  
                for rect in rects:
                    height = rect.get_height()
                    s.annotate(f'{height}%',
                                    xy=(rect.get_x() + rect.get_width() / 2, height),
                                    xytext=(0, 3),  # 3 points vertical offset
                                    textcoords="offset points",
                                    ha='center', va='bottom')
                j+=1

In [ ]:
df = df[df['name'] == 'DE1'][:]

In [ ]:
print('[20] : Devices and the Number of times their ACS conenctivity was reported to be down : \n')
df2 = df[~df['srv1'].astype('bool')][['partition_date', 'di8', 'dt']].reset_index(drop=True).groupby(['di8', 'partition_date']).count()
df2.rename(columns = {'dt' : 'Number'}).rename_axis(index={ 'di8' : 'RSN', 'partition_date' : 'Date'}).stack().unstack(level=1).fillna('.')

In [ ]:
print('[21] : Devices and the Number of times their NTP Sync was reported to have failed : \n')
df2 = df[~df['srv2'].astype(bool)][['partition_date', 'di8', 'dt']].reset_index(drop=True).groupby(['di8', 'partition_date']).count()
df2.rename(columns = {'dt' : 'Number'}).rename_axis(index={ 'di8' : 'RSN', 'partition_date' : 'Date'}).stack().unstack(level = 1).fillna('.')

In [ ]:
print('[22] : Devices and the Number of times their WAN IPv6 was down :\n')
df2 = df[~df['wan3'].astype('bool')][['partition_date', 'di8', 'dt']].reset_index(drop=True).groupby(['di8', 'partition_date']).count()
df2.rename(columns = {'dt' : 'Number'}).rename_axis(index={ 'di8' : 'RSN', 'partition_date' : 'Date'}).stack().unstack(level=1).fillna('.')

In [ ]:
'''
explode expands a column containing lists to a column with individual list elements. All other elements are duplicated
including the index.
drop simply removes a column.
Axis along which the function is applied:
0 or ‘index’: apply function to each column. like .sum, .count
1 or ‘columns’: apply function to each row. (or columns = 'wi18')

pd.Series converts individual dictionary into a series which is converted to a df by the apply.
concat joins the two dfs column wise by aligning the indexes
'''
dfe = df.explode('wi17').explode('wi18')
df2 = pd.concat([dfe.reset_index(drop=True), 
                 pd.json_normalize(dfe['wi17'].fillna({i:{} for i in dfe.index}))],
                 axis=1).reset_index()
df3 = df2[['di8', 'partition_date', 'dt', 'mac', 'rssi', 'txrate', 'max_txrate', 'nxn', 'txbytes', 'rxbytes', 'di4']].copy()

In [ ]:
pd.concat([dfe.reset_index(drop=True), pd.json_normalize(dfe['wi17'].fillna({i:{} for i in dfe.index}))], axis=1).reset_index()

In [ ]:
pd.concat([dfe.drop('wi17', axis=1), pd.DataFrame(dfe['wi17'].fillna({i:{} for i in dfe.index}).to_list(), index=dfe.index)], axis=1)

In [ ]:
def poorcov(d):
    return ((sum(d['rssi'] < -85) / len(d.rssi)) >= 0.2*len(d.rssi))
df4= df3.groupby(['partition_date', 'di8', 'mac']).apply(poorcov).reset_index().rename(columns = {0:'client_flag'})

In [ ]:
def badband(d):
    return(sum(d.client_flag) >= 0.5*len(d.client_flag))
df5 = df4.groupby(['partition_date', 'di8']).apply(badband).reset_index().rename(columns = {0:'Flag'})
print('[23] : Devices with poor 2.4GHz WiFi coverage (low rssi) : \n')
df5.groupby('partition_date')['Flag'].sum().plot(kind='bar', color = 'indianred', figsize=(9,5),\
                                                 title = '[24] : Devices with poor 2.4GHz WiFi coverage (low rssi) :',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
def poortx(d):
    if d.di4.unique()[0] in ['JCO110', 'JCO500', 'HG260ES']:
        return ((sum(d['max_txrate'] < 19.5) / len(d.max_txrate)) >= 0.2*len(d.max_txrate))
    else:
        return ((sum(d['txrate'] < 19.5) / len(d.txrate)) >= 0.2*len(d.txrate))
df4= df3.groupby(['partition_date', 'di8', 'mac']).apply(poortx).reset_index().rename(columns = {0:'client_flag'})
df5 = df4.groupby(['partition_date', 'di8']).apply(badband).reset_index().rename(columns = {0:'Flag'})
df5.groupby('partition_date')['Flag'].sum().plot(kind='bar', color = 'indianred', figsize=(9,5),\
                                                 title = '[24] : Devices with poor txrates on 2.4GHz band (bad band)',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
pd.set_option('display.memory_usage', True, 'compute.use_numba', False)

In [ ]:
def numChanges(x):
    return sum(x.iloc[:-1] != x.shift(-1).iloc[:-1])

df4=df3.groupby(['partition_date', 'di8', 'mac']).agg({'nxn' : numChanges})
df4.rename(columns = {'nxn' : 'MIMO switches'}, inplace =True)
df4.groupby('partition_date')['MIMO switches'].sum().plot(kind='bar', color = 'salmon', figsize=(9,5),\
                                                 title = '[25] : Number of mimo config switches in a day vs Date',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
print('[26] : Top 5 devices that exhibited most switches: \n')
df4.groupby(['partition_date', 'di8']).sum().sort_values(by='MIMO switches', ascending=False)[:6]

In [ ]:
df4 = pd.concat([dfe.reset_index(drop=True), 
                 pd.json_normalize(dfe['wi18'].fillna({i:{} for i in dfe.index}))],
                 axis=1).reset_index()
df5 =df4[['di8', 'partition_date', 'dt', 'mac', 'rssi', 'txrate', 'max_txrate', 'nxn', 'txbytes', 'rxbytes', 'di4']][:]

In [ ]:
def poorcov(d):
    return ((sum(d['rssi'] < -85) / len(d.rssi)) >= 0.2*len(d.rssi))
df6= df5.groupby(['partition_date', 'di8', 'mac']).apply(poorcov).reset_index().rename(columns = {0:'client_flag'})
def badband(d):
    return(sum(d.client_flag) >= 0.5*len(d.client_flag))
df7 = df6.groupby(['partition_date', 'di8']).apply(badband).reset_index().rename(columns = {0:'Flag'})
df7.groupby('partition_date')['Flag'].sum().plot(kind='bar', color = 'indianred', figsize=(9,5),\
                                                 title = '[27] : Devices with poor 5GHz WiFi coverage (low rssi) :',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
def poortx(d):
    if d.di4.unique()[0] in ['JCO110', 'JCO500', 'HG260ES']:
        return ((sum(d['max_txrate'] < 19.5) / len(d.max_txrate)) >= 0.2*len(d.max_txrate))
    else:
        return ((sum(d['txrate'] < 19.5) / len(d.txrate)) >= 0.2*len(d.txrate))
df6= df5.groupby(['partition_date', 'di8', 'mac']).apply(poortx).reset_index().rename(columns = {0:'client_flag'})
df7 = df6.groupby(['partition_date', 'di8']).apply(badband).reset_index().rename(columns = {0:'Flag'})
df7.groupby('partition_date')['Flag'].sum().plot(kind='bar', color = 'indianred', figsize=(9,5),\
                                                 title = '[28]  : Devices with poor txrates on 5GHz band (bad band) : :',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
def numChanges(x):
    return sum(x.iloc[:-1] != x.shift(-1).iloc[:-1])

df6=df5.groupby(['partition_date', 'di8', 'mac']).agg({'nxn' : numChanges})
df6.rename(columns = {'nxn' : 'MIMO switches'}, inplace =True)
df6.groupby('partition_date')['MIMO switches'].sum().plot(kind='bar', color = 'salmon', figsize=(9,5),\
                                                 title = '[29] : Number of mimo config switches in a day vs Date',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
#df6.query("partition_date == '2021-03-23'")
print('[30] : Top 5 devices that exhibited most switches: \n')
df6.groupby(['partition_date', 'di8']).sum().sort_values(by='MIMO switches', ascending=False)[:6]

In [ ]:
df3.loc[:, 'band'] = '2.4GHz'
df5.loc[:, 'band'] = '5GHz'
df7= df5.merge(df3, how='outer')
def switchcheck(d):
    return(len(d.band.unique()) >1)
df8= df7.groupby(['di8', 'partition_date', 'mac']).apply(switchcheck).reset_index().rename(columns = {0:'shift'})
print('[31] : Devices with clients that switched bands: \n')
df8.groupby('partition_date')['shift'].sum().plot(kind='bar', color = 'lightgreen', figsize=(9,5),\
                                                 title = '[31] : Number of times a client switched bands (between 2.4 & 5) vs Date :',\
                                                 xlabel='Dates', ylabel='Count of devices')
print('')

In [ ]:
print('[32] : Devices exhibiting the most number of switches from the above plot: \n')
df8.groupby('di8')['shift'].sum().reset_index().sort_values(by='shift', ascending =False).reset_index(drop=True)[:6]

In [ ]:
!jupyter nbconvert "Field_rollout_checks.ipynb" --no-input --to html --output f"{df.di4.unique()[0]}_{df.di5.unique()[0][1:]}_Rolloutcheck".html

In [ ]:
#_ih[-500:]

In [ ]:
time2 = time()
print(f'{time2-time1} seconds taken. \nEND of Report')